In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
# XML-Lib laden

import json

In [33]:
# url Ergebnisse
url_ergebnisse = "./test_daten/Gesamtergebnis_LWT19_WahlbezirkeDD.csv"
# url Daten Wahllokale
url_wahllokale = "./test_daten/Wahllokale_Dresden.csv"

# Ergebnisse auf Wahlbezirkebene
df = pd.read_csv(url_ergebnisse, encoding="cp1252", sep=";")
# Wahllokale
df_wl = pd.read_csv(url_wahllokale, header=1)

In [34]:
df

,Wahlbezirk,Wahlkreis,Wahlberechtigte,Wähler,gültige_1,CDU_1,DIE LINKE_1,SPD_1,AfD_1,GRÜNE_1,...,PIRATEN_2 Deutschland,Die PARTEI_2,BüSo_2,ADPM_2,Blaue #TeamPetry_2,KPD_2,ÖDP_2,Die Humanisten_2,PDV_2,Gesundheitsforschung_2
0,1100,45.0,1333,506,501,139,110,26,85,93,...,2,17,0,0,2,0,1,6,0,3
1,2100,45.0,1332,498,492,87,145,31,79,100,...,9,19,1,0,0,1,5,2,0,2
2,2200,45.0,1700,658,646,161,152,33,141,100,...,6,15,1,0,2,2,7,9,1,2
3,2400,45.0,1390,489,485,134,78,38,138,52,...,3,7,1,0,1,0,4,0,0,4
4,3100,45.0,1219,501,492,121,85,34,111,84,...,6,17,2,2,1,0,2,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,98001,46.0,0,702,694,273,72,59,163,67,...,2,9,1,0,2,1,3,2,2,4
501,98002,46.0,0,877,871,320,120,67,213,92,...,3,19,1,1,5,0,3,2,2,5
502,98003,46.0,0,841,836,340,80,67,176,120,...,4,4,0,0,3,2,4,2,2,4
503,99010,46.0,0,921,909,393,89,68,218,76,...,3,7,1,0,2,0,1,0,0,8


# Data-Cleaning

## richtige Datentypen vergebne, führende Null Wahlbezirke einbauen


In [3]:
def change_wb_id(num):
    if len(str(num))== 4: 
        return "0" + str(num)
    else: 
        return str(num)
    


In [4]:
df["Wahlbezirk"] = df["Wahlbezirk"].map(change_wb_id)
df["Wahlkreis"] = df["Wahlkreis"].astype(str)
df = df.fillna(0)

## neues Ergebnis-Dataframe berechnen

In [25]:
def create_wb_dataframe(df): 
    
    cols = ["Wahlbezirk", "Wahlkreis", "Wahlberechtige", "Wähler", \
            'Wahlbeteiligung',
            
           '1_Stimmen AfD', '1_Stimmen CDU', '1_Stimmen FDP',
           '1_Stimmen GRÜNE', '1_Stimmen LINKE', '1_Stimmen SPD',
           '1_Stimmen Ungültige Stimmen', '1_Stimmen andere Parteien',
           '1_Stimmen gültige Stimmen', '1_Anteil AfD', '1_Anteil CDU',
           '1_Anteil FDP', '1_Anteil GRÜNE', '1_Anteil LINKE', '1_Anteil SPD',
           '1_Anteil Ungültige Stimmen', '1_Anteil andere Parteien',
           '1_Anteil gültige Stimmen', \
            
            '2_Stimmen AfD', '2_Stimmen CDU',
           '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD',
           '2_Stimmen Ungültige Stimmen', '2_Stimmen andere Parteien',
           '2_Stimmen gültige Stimmen', '2_Anteil AfD', '2_Anteil CDU',
           '2_Anteil FDP', '2_Anteil GRÜNE', '2_Anteil LINKE', '2_Anteil SPD',
           '2_Anteil Ungültige Stimmen', '2_Anteil andere Parteien',
           '2_Anteil gültige Stimmen']
            
    new_df = pd.DataFrame(columns=cols)
    
    for row in df.itertuples(): 
        wb = row.Wahlbezirk
        wk = row.Wahlkreis
        wberechtigte = row.Wahlberechtigte
        wähler = row.Wähler
        try: 
            wahlbeteiligung = round(wähler/wberechtigte * 100,1)
        except: 
            print("Fehler")
            print(f"Wahlbezirk {wb}, Wahlberechtigte: {wberechtigte}, wähler: {wähler}")
        
        # Erststimmen
        e_stimmen_afd = row.AfD_1
        e_stimmen_cdu = row.CDU_1
        e_stimmen_spd = row.SPD_1
        e_stimmen_linke = row._7
        e_stimmen_gruene = row.GRÜNE_1
        e_stimmen_fdp = row.FDP_1
        e_stimmen_gültig = row.gültige_1
        e_stimmen_ungültig = wähler - e_stimmen_gültig
        
        e_stimmen_andere = e_stimmen_gültig - e_stimmen_afd -  e_stimmen_cdu - e_stimmen_fdp -\
                             e_stimmen_gruene -  e_stimmen_linke -  e_stimmen_spd
        e_anteil_afd = round(e_stimmen_afd / e_stimmen_gültig * 100,1)
        e_anteil_cdu = round(e_stimmen_cdu / e_stimmen_gültig * 100,1)
        e_anteil_fdp = round(e_stimmen_fdp / e_stimmen_gültig * 100,1)
        e_anteil_spd = round(e_stimmen_spd / e_stimmen_gültig * 100,1)
        e_anteil_gruene = round(e_stimmen_gruene / e_stimmen_gültig * 100,1)
        e_anteil_linke = round(e_stimmen_linke / e_stimmen_gültig * 100,1)
        e_anteil_andere = round(e_stimmen_andere / e_stimmen_gültig * 100,1)
        e_anteil_ungültige_stimmen = round(e_stimmen_ungültig / wähler * 100, 1)
        e_anteil_gültige_stimmen = round(e_stimmen_gültig / wähler * 100,1)
        
        
        # Zweitstimmen
        z_stimmen_afd = row.AfD_2
        z_stimmen_cdu = row.CDU_2
        z_stimmen_spd = row.SPD_2
        z_stimmen_linke = row._16
        z_stimmen_gruene = row.GRÜNE_2
        z_stimmen_fdp = row.FDP_2
        z_stimmen_gültig = row.gültige_2
        z_stimmen_ungültig = wähler - z_stimmen_gültig
        
        z_stimmen_andere = z_stimmen_gültig - z_stimmen_afd -  z_stimmen_cdu - z_stimmen_fdp -\
                             z_stimmen_gruene -  z_stimmen_linke -  z_stimmen_spd
        z_anteil_afd = round(z_stimmen_afd / z_stimmen_gültig * 100,1)
        z_anteil_cdu = round(z_stimmen_cdu / z_stimmen_gültig * 100,1)
        z_anteil_fdp = round(z_stimmen_fdp / z_stimmen_gültig * 100,1)
        z_anteil_spd = round(z_stimmen_spd / z_stimmen_gültig * 100,1)
        z_anteil_gruene = round(z_stimmen_gruene / z_stimmen_gültig * 100,1)
        z_anteil_linke = round(z_stimmen_linke / z_stimmen_gültig * 100,1)
        z_anteil_andere = round(z_stimmen_andere / z_stimmen_gültig * 100,1)
        z_anteil_ungültige_stimmen = round(z_stimmen_ungültig / wähler * 100, 1)
        z_anteil_gültige_stimmen = round(z_stimmen_gültig / wähler * 100,1)
    
        
        
        data_ser = [wb, wk, wberechtigte, wähler, wahlbeteiligung, 
                    # Erststimmen
                    e_stimmen_afd, e_stimmen_cdu, e_stimmen_fdp, e_stimmen_gruene, e_stimmen_linke, e_stimmen_spd, \
                    e_stimmen_ungültig, e_stimmen_andere, e_stimmen_gültig, \
                    e_anteil_afd, e_anteil_cdu, e_anteil_fdp, e_anteil_gruene, e_anteil_linke, e_anteil_spd, \
                     e_anteil_ungültige_stimmen, e_anteil_andere, e_anteil_gültige_stimmen, \
                    #Zweitstimmen
                    z_stimmen_afd, z_stimmen_cdu, z_stimmen_fdp, z_stimmen_gruene, z_stimmen_linke, z_stimmen_spd, \
                   z_stimmen_ungültig, z_stimmen_andere, z_stimmen_gültig, \
                    z_anteil_afd, z_anteil_cdu, z_anteil_fdp, z_anteil_gruene, z_anteil_linke, z_anteil_spd, \
                    z_anteil_ungültige_stimmen, z_anteil_andere, z_anteil_gültige_stimmen
                   ]
        
        new_ser = pd.Series(data_ser, index=cols)
        new_df = new_df.append(new_ser, ignore_index=True)

    return new_df
        
        
        
        
        
df_results = create_wb_dataframe(df)        

Fehler
Wahlbezirk 01000, Wahlberechtigte: 0, wähler: 462
Fehler
Wahlbezirk 02001, Wahlberechtigte: 0, wähler: 785
Fehler
Wahlbezirk 02002, Wahlberechtigte: 0, wähler: 480
Fehler
Wahlbezirk 03001, Wahlberechtigte: 0, wähler: 617
Fehler
Wahlbezirk 03002, Wahlberechtigte: 0, wähler: 963
Fehler
Wahlbezirk 04001, Wahlberechtigte: 0, wähler: 1252
Fehler
Wahlbezirk 04002, Wahlberechtigte: 0, wähler: 783
Fehler
Wahlbezirk 05001, Wahlberechtigte: 0, wähler: 885
Fehler
Wahlbezirk 05002, Wahlberechtigte: 0, wähler: 781
Fehler
Wahlbezirk 06001, Wahlberechtigte: 0, wähler: 516
Fehler
Wahlbezirk 06002, Wahlberechtigte: 0, wähler: 972
Fehler
Wahlbezirk 06003, Wahlberechtigte: 0, wähler: 888
Fehler
Wahlbezirk 07001, Wahlberechtigte: 0, wähler: 995
Fehler
Wahlbezirk 07002, Wahlberechtigte: 0, wähler: 780
Fehler
Wahlbezirk 07003, Wahlberechtigte: 0, wähler: 902
Fehler
Wahlbezirk 11001, Wahlberechtigte: 0, wähler: 1103
Fehler
Wahlbezirk 11002, Wahlberechtigte: 0, wähler: 962
Fehler
Wahlbezirk 11003, Wahl

## richtige Formate vergeben

In [28]:
df_results[["1_Stimmen AfD", "1_Stimmen CDU", "1_Stimmen FDP", "1_Stimmen GRÜNE", "1_Stimmen LINKE", \
            "1_Stimmen SPD", "1_Stimmen Ungültige Stimmen", "1_Stimmen andere Parteien", "1_Stimmen gültige Stimmen",  \
           "2_Stimmen AfD", "2_Stimmen CDU", "2_Stimmen FDP", "2_Stimmen GRÜNE", "2_Stimmen LINKE", \
           "2_Stimmen SPD", "2_Stimmen Ungültige Stimmen", "2_Stimmen andere Parteien", "2_Stimmen gültige Stimmen" ]] = \
df_results[["1_Stimmen AfD", "1_Stimmen CDU", "1_Stimmen FDP", "1_Stimmen GRÜNE", "1_Stimmen LINKE", \
            "1_Stimmen SPD", "1_Stimmen Ungültige Stimmen", "1_Stimmen andere Parteien", "1_Stimmen gültige Stimmen" ,\
           "2_Stimmen AfD", "2_Stimmen CDU", "2_Stimmen FDP", "2_Stimmen GRÜNE", "2_Stimmen LINKE", \
           "2_Stimmen SPD", "2_Stimmen Ungültige Stimmen", "2_Stimmen andere Parteien", "2_Stimmen gültige Stimmen" ]].astype(int)

In [29]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Wahlbezirk                   505 non-null    object 
 1   Wahlkreis                    505 non-null    object 
 2   Wahlberechtige               505 non-null    object 
 3   Wähler                       505 non-null    object 
 4   Wahlbeteiligung              505 non-null    float64
 5   1_Stimmen AfD                505 non-null    int32  
 6   1_Stimmen CDU                505 non-null    int32  
 7   1_Stimmen FDP                505 non-null    int32  
 8   1_Stimmen GRÜNE              505 non-null    int32  
 9   1_Stimmen LINKE              505 non-null    int32  
 10  1_Stimmen SPD                505 non-null    int32  
 11  1_Stimmen Ungültige Stimmen  505 non-null    int32  
 12  1_Stimmen andere Parteien    505 non-null    int32  
 13  1_Stimmen gültige St

## Gewinner Zweitstimmen festlegen


In [30]:
df_results["2_Gewinner"] = df_results[['2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD']].idxmax(axis=1)
df_results["2_Gewinner"] = df_results["2_Gewinner"].str.split(" ")
df_results["2_Gewinner"] = df_results["2_Gewinner"].map(lambda x: x[1])

In [31]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Wahlbezirk                   505 non-null    object 
 1   Wahlkreis                    505 non-null    object 
 2   Wahlberechtige               505 non-null    object 
 3   Wähler                       505 non-null    object 
 4   Wahlbeteiligung              505 non-null    float64
 5   1_Stimmen AfD                505 non-null    int32  
 6   1_Stimmen CDU                505 non-null    int32  
 7   1_Stimmen FDP                505 non-null    int32  
 8   1_Stimmen GRÜNE              505 non-null    int32  
 9   1_Stimmen LINKE              505 non-null    int32  
 10  1_Stimmen SPD                505 non-null    int32  
 11  1_Stimmen Ungültige Stimmen  505 non-null    int32  
 12  1_Stimmen andere Parteien    505 non-null    int32  
 13  1_Stimmen gültige St

## Colum Wahlbezirk für Datawrapper in Index verwandeln

In [32]:
# column Wahlbezirk muss in wb umbenannt werden, damit datawrapper matchen kann
df_results.rename(columns={"Wahlbezirk":"wb"}, inplace=True)
# column wb zum Index machen, damit datawrapper es auslesen kann
df_results.set_index('wb', inplace=True)

df_results.to_csv("./data/wb_dresden_testdaten.csv")

In [12]:
# Offizielle Listennummer mit Parteiname BTW 2021
parteien_dict = {"1":"AfD", "2":"CDU", "3":"LINKE", "4":"SPD", "5":"FDP", "6":"GRÜNE", \
                # kleinere Parteien (sonstige)
                "7":"Tierschutzpartei", "8":"Die PARTEI", "9":"NPD", "10":"FREIE WÄHLER", \
                 "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
                 "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
                 "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
                 # kleinere Parteien ohne Landesliste
                 "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
                 "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
                 "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
                 "33": "Internationalistisches Bündnis", "34": "Internationalistisches Bündnis", \
                 "35": "Internationalistisches Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
                 "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"REICHEL",\
                 "42":"Röhder", "43":"UNO - Menschrecht - FRIEDEN für Alle", "44":"WIR sind LEIPZIGER", \
                 "45":"www.zeitkaufhaus.de Martin Bayer", \
                 # Bezeichner für gültige und ungültige Stimmen
                "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

                }

In [20]:
for row in df.itertuples(): 
    print(row)

Pandas(Index=0, Wahlbezirk='01100', Wahlkreis='45.0', Wahlberechtigte=1333, Wähler=506, gültige_1=501, CDU_1=139, _7=110, SPD_1=26, AfD_1=85, GRÜNE_1=93, FDP_1=32, _12=16.0, BüSo_1=0.0, gültige_2=504, CDU_2=140, _16=83, SPD_2=35, AfD_2=85, GRÜNE_2=83, NPD_2=1, FDP_2=37, _22=5, Tierschutzpartei_2=4, _24=2, _25=17, BüSo_2=0, ADPM_2=0, _28=2, KPD_2=0, ÖDP_2=1, _31=6, PDV_2=0, Gesundheitsforschung_2=3)
Pandas(Index=1, Wahlbezirk='02100', Wahlkreis='45.0', Wahlberechtigte=1332, Wähler=498, gültige_1=492, CDU_1=87, _7=145, SPD_1=31, AfD_1=79, GRÜNE_1=100, FDP_1=21, _12=26.0, BüSo_1=3.0, gültige_2=491, CDU_2=87, _16=113, SPD_2=43, AfD_2=74, GRÜNE_2=96, NPD_2=2, FDP_2=24, _22=8, Tierschutzpartei_2=5, _24=9, _25=19, BüSo_2=1, ADPM_2=0, _28=0, KPD_2=1, ÖDP_2=5, _31=2, PDV_2=0, Gesundheitsforschung_2=2)
Pandas(Index=2, Wahlbezirk='02200', Wahlkreis='45.0', Wahlberechtigte=1700, Wähler=658, gültige_1=646, CDU_1=161, _7=152, SPD_1=33, AfD_1=141, GRÜNE_1=100, FDP_1=29, _12=26.0, BüSo_1=4.0, gültige_

In [17]:
df

Index(['Wahlbezirk', 'Wahlkreis', 'Wahlberechtigte', 'Wähler', 'gültige_1',
       'CDU_1', 'DIE LINKE_1', 'SPD_1', 'AfD_1', 'GRÜNE_1', 'FDP_1',
       'Die PARTEI_1', 'BüSo_1', 'gültige_2', 'CDU_2', 'DIE LINKE_2', 'SPD_2',
       'AfD_2', 'GRÜNE_2', 'NPD_2', 'FDP_2', 'FREIE WÄHLER_2',
       'Tierschutzpartei_2', 'PIRATEN_2 Deutschland', 'Die PARTEI_2', 'BüSo_2',
       'ADPM_2', 'Blaue #TeamPetry_2', 'KPD_2', 'ÖDP_2', 'Die Humanisten_2',
       'PDV_2', 'Gesundheitsforschung_2'],
      dtype='object')

In [ ]:
Index(['AGS', 'Ort', '1_Stimmen AfD', '1_Stimmen CDU', '1_Stimmen FDP',
       '1_Stimmen GRÜNE', '1_Stimmen LINKE', '1_Stimmen SPD',
       '1_Stimmen Ungültige Stimmen', '1_Stimmen andere Parteien',
       '1_Stimmen gültige Stimmen', '1_Anteil AfD', '1_Anteil CDU',
       '1_Anteil FDP', '1_Anteil GRÜNE', '1_Anteil LINKE', '1_Anteil SPD',
       '1_Anteil Ungültige Stimmen', '1_Anteil andere Parteien',
       '1_Anteil gültige Stimmen', '2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD',
       '2_Stimmen Ungültige Stimmen', '2_Stimmen andere Parteien',
       '2_Stimmen gültige Stimmen', '2_Anteil AfD', '2_Anteil CDU',
       '2_Anteil FDP', '2_Anteil GRÜNE', '2_Anteil LINKE', '2_Anteil SPD',
       '2_Anteil Ungültige Stimmen', '2_Anteil andere Parteien',
       '2_Anteil gültige Stimmen', 'WKNr', 'Stimmen Wahlberechtigte',
       'Stimmen Wähler', 'Anteil Wahlberechtigte', 'Anteil Wähler',
       '2017_AfD_%', '2017_CDU_%', '2017_LINKE_%', '2017_SPD_%', '2017_FDP_%',
       '2017_GRÜNE_%', '2017_andere_%'],
      dtype='object')